In [ ]:
hviimport os
import s3fs
import pandas as pd

os.chdir("../")

from src.core.configs import settings, fs
from src.dataloader.loader import CustomImageDataset, MaskDataloaders

In [ ]:
df_classes = pd.read_csv(fs.open(f"{settings.s3_prefix}_classes.csv")).drop(axis=1, columns=[" without_mask"])

df_train = df_classes.groupby(' with_mask', group_keys=False).apply(lambda x: x.sample(frac=0.7))

df_validation_test = df_classes[~df_classes.index.isin(df_train.index)]

df_validation = df_validation_test.groupby(' with_mask', group_keys=False).apply(lambda x: x.sample(frac=0.7))

df_test = df_validation_test[~df_validation_test.index.isin(df_validation.index)]

In [ ]:
len(MaskDataloaders(settings.s3_prefix, [0.7, 0.2, 0.1]).getDataloaders()[2])